# ICE ONLINE GAMES: Descubriendo el éxito a través de los datos.

La tienda online Ice vende videojuegos globalmente, teniendo acceso a datos históricos de ventas, reseñas de usuarios y expertos, géneros y plataformas de juegos el objetivo es identificar patrones que determinen el éxito de un juego, para detectar proyectos prometedores y planificar campañas publicitarias. Los datos disponibles se remontan a 2016 pronosticando ventas para 2017. El dataset incluye la clasificación ESRB de cada juego, que evalúa su contenido y asigna una clasificación de edad. El enfoque es aprovechar estos datos para tomar decisiones informadas sobre la estrategia de ventas y publicidad.

## Información general 

### Librerías

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats as st

### Cargar datos

In [ ]:
df_games= pd.read_csv('/datasets/games.csv')

In [ ]:
df_games.info()

In [ ]:
df_games.head(10)

## Preparar los datos

### Reemplazar nombres de columnas

In [ ]:
new_col_names = []
for old_name in df_games.columns:
    name_lowered = old_name.lower()
    new_col_names.append(name_lowered)
df_games.columns = new_col_names 

print(df_games.columns)

### Convertir tipos de datos

In [ ]:
df_games['year_of_release'] = df_games['year_of_release'].astype('Int64')
df_games['user_score'] = pd.to_numeric(df_games['user_score'], errors='coerce')
print(df_games.dtypes)

### Descripción de cambios

Una vez que tenemos el nombre de las columnas en minúsculas, hacemos cambios en el tipo de datos de algunas columnas como *year_of_release* (año de lanzamiento) ya que sería mas apropiado manejarlo como entero (*int*) para eliminar los decimales y reflejar que son valores de años enteros. Para la columna *user_score* (puntuación del usuario) se convierten los valores en la columna a tipo numérico.

### Tratar valores ausentes

In [ ]:
df_games['name'] = df_games['name'].fillna(' ')
df_games['genre'] = df_games['genre'].fillna(' ')
df_games.info()

Para determinar qué columnas debemos tratar en cuanto a valores ausentes, es importante analizar la naturaleza de los datos y cómo serán utilizados en nuestro análisis; en este caso se han dejado en blanco los valores ausentes de la columna *name* (nombre) y *genre* (género) solo para garantizar que todas las filas tengan valores no nulos, facilitar el procesamiento posterior de los datos. 

Los datos podrían estar ausentes en estas columnas debido a una variedad de factores, que van desde errores en la entrada de datos, la falta registros o datos públicos, las reseñas o datos de juegos antiguos o de aquellos que no han tenido el suficiente exito para recibir atención o incluso la información podria estar ausente debido a políticas de privacidad. 

Para manejar los valores "TBD" (To Be Determined) en la columna *user_score* (puntuación del usuario) previamente se convirtieron los valores a tipo numérico, utilizando el parametro *'errors='coerce'* para mantener "TBD" en NaN y asi la columna siga teniendo valores de tipo numérico para análisis posteriores.

### Ventas totales

In [ ]:
df_games['total_sales'] = df_games['na_sales'] + df_games['eu_sales'] + df_games['jp_sales'] + df_games['other_sales']
columns=['name','platform','year_of_release','total_sales']
df_games[columns]

In [ ]:
df_games['critic_score'] = df_games['critic_score'].fillna('Unknown')
df_games['user_score'] = df_games['user_score'].fillna('Unknown')

df_games['critic_score'] = pd.to_numeric(df_games['critic_score'], errors='coerce')
df_games['user_score'] = pd.to_numeric(df_games['user_score'], errors='coerce')

print(df_games.dtypes)

## Analizar los datos

### Número de juegos lanzados por año

In [ ]:
games_per_year = df_games['year_of_release'].value_counts().sort_index()

plt.figure(figsize=(8,6))
games_per_year.plot(marker='*', linestyle='-', color= 'purple', grid= True)
plt.title('Número de juegos lanzados por año')
plt.xlabel('Año de lanzamiento')
plt.ylabel('Número de juegos')
plt.show()

Al observar el gráfico resultante, podemos determinar que una alza en el numero de lanzamientos de videojuegos desde poco antes del año 2000, pero en los años 2008 y 2009 hubo un número significativamente mayor de juegos lanzados. 

### Ventas por plataforma

In [ ]:
# Calcular y ordenar las ventas totales por plataforma (Top 5)
platform_total_sales = df_games.groupby('platform')['total_sales'].sum()
top_platforms = platform_total_sales.sort_values(ascending=False).head(5)

# Filtrar y agrupar el DataFrame para incluir solo las filas de las plataformas más vendidas
df_top_platforms = df_games[df_games['platform'].isin(top_platforms.index)]
yearly_platform_sales = df_top_platforms.groupby(['year_of_release', 'platform'])['total_sales'].sum().unstack()

# Visualizar la distribución de ventas para las plataformas más vendidas
yearly_platform_sales.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.title('Distribución de ventas por año y plataforma')
plt.xlabel('Año de lanzamiento')
plt.ylabel('Ventas totales (en millones)')
plt.legend(title='Plataforma')
plt.grid(axis='y')
plt.show()

En el gráfico podemos observar el comportamiento de las ventas de las principales plataformas a lo largo de los años, las ventas de PlayStation 2 mostraron un gran numero de ventas la mitad de la decada de los 2000, luego entro Nintendo arrebatandole poco mas de la mitad de las ventas, tambien entra en juego Microsoft con Xbox, asi PS2 termina por dejarle paso a su sucesor el PlayStation 3 entre 2009 y 2010, lo que nos lleva observar el comportamiento de vida de una plataforma, Nintendo comienza firme en 2005 en sus dos plataformas pero para 2012 sus ventas ya son minimas, Xbox360 se mantiene un poquito mas en la batalla comenzando en 2006 hasta 2015. En conclusión la vida útil de una plataforma oscila entre los 5-6 años, pero otros factores pueden alterar este comportamiento, como el genero de los videojuegos, entre otros.  

### Determinar periodo

Considerando que la industria de los videojuegos ha experimentado cambios significativos en las últimas décadas, podríamos enfocarnos en un período más reciente que aún nos proporcione suficiente historia y datos relevantes. Por ejemplo, podríamos tomar datos desde el año 2000 hasta 2016. Este período de 16 años nos brinda una visión significativa de las tendencias y patrones en la industria de los videojuegos, permitiéndonos capturar cambios significativos mientras mantenemos la relevancia de los datos para el año objetivo, 2017.

### Trabajar con datos relevantes

In [ ]:
df_filtered = df_games.loc[(df_games['year_of_release'] >= 2000) & (df_games['year_of_release'] <= 2016)]
df_filtered.head(10)

### ¿Qué plataformas son líderes en ventas?

In [ ]:
# Plataformas lideres en ventas
platform_total_sales = df_filtered.groupby('platform')['total_sales'].sum()
platform_total_sales_sorted = platform_total_sales.sort_values(ascending=False)

print("Plataformas líderes en ventas:")
print(platform_total_sales_sorted.head())

# Calcular el promedio de las ventas totales por plataforma 
platform_sales_mean = df_filtered.groupby(['platform'])['total_sales'].mean()

# Identificar las plataformas potencialmente rentables 
potentially_profitable_platforms = platform_sales_mean.sort_values(ascending=False).head(5)

# Identificar las plataformas en declive 
declining_platforms = platform_sales_mean.sort_values().head(5)

print("\nPlataformas potencialmente rentables:")
print(potentially_profitable_platforms)
print("\nPlataformas en declive:")
print(declining_platforms)

### Diagrama de caja

In [ ]:
plt.figure(figsize=(12, 6))
plt.boxplot([df_filtered[df_filtered['platform'] == platform]['total_sales'] for platform in df_filtered['platform'].unique()],labels=df_filtered['platform'].unique())
plt.title('Ventas globales de juegos por plataforma')
plt.xlabel('Plataforma')
plt.ylabel('Ventas globales (en millones)')
plt.xticks(rotation=90)
plt.grid(True)
plt.show()

### Reseñas 

In [ ]:
df_filtered.info()

In [ ]:
df_ds = df_filtered[df_filtered['platform'] == 'DS']
df_ds = df_ds[df_ds['critic_score'] != 'Unknown']
df_ds = df_ds[df_ds['user_score'] != 'Unknown']

df_ds = df_ds.dropna(subset=['critic_score', 'user_score'])

plt.figure(figsize=(10, 6))
plt.scatter(df_ds['critic_score'], df_ds['total_sales'], color='blue', label='Critic Score')
plt.scatter(df_ds['user_score'], df_ds['total_sales'], color='red', label='User Score')
plt.title('Ventas vs. Reseñas para la plataforma DS')
plt.xlabel('Reseñas')
plt.ylabel('Ventas (en millones)')
plt.legend()
plt.grid(True)
plt.show()

correlation_critic = df_ds['critic_score'].corr(df_ds['total_sales'])
correlation_user = df_ds['user_score'].corr(df_ds['total_sales'])

print("Correlación entre las reseñas de críticos y las ventas:", correlation_critic)
print("Correlación entre las reseñas de usuarios y las ventas:", correlation_user)

En el gráfico de dispersión observamos una distribución dispersa de puntos, lo que sugiere que no hay una relación lineal clara entre las reseñas y las ventas, pero parece haber una tendencia general de que los juegos con reseñas más altas tienden a tener ventas más altas, especialmente en el caso de las reseñas de críticos.

La correlación entre las reseñas de críticos y las ventas es moderada positiva, es decir, hay una relación significativa pero no es extremadamente fuerte. La correlación entre las reseñas de usuarios y las ventas es positiva débil aunque existe una relación, es menos fuerte que la correlación entre las reseñas de críticos y las ventas.

### Juegos en diferentes plataformas

In [ ]:
selected_games = ['FIFA 14', 'Grand Theft Auto V', 'Star Wars The Clone Wars', "Harry Potter and the Sorcerer's Stone", 'F1 2011', 'Need for Speed Carbon']

df_selected_games = df_filtered[df_filtered['name'].isin(selected_games)].dropna(subset=['total_sales'])

game_sales_by_platform = df_selected_games.groupby(['name', 'platform'])['total_sales'].sum().unstack()

In [ ]:
game_sales_by_platform.plot(kind='bar', figsize=(10, 6), stacked=True, rot=45)
plt.title('Ventas de juegos en diferentes plataformas')
plt.xlabel('Juego')
plt.ylabel('Ventas (en millones)')
plt.legend(title='Plataforma')
plt.show()

Hay muchos juegos que son exclusivos de cada plataforma, otros que no son compatibles pero al observar estos tres juegos multiplataforma podemos observar que las ventas son mayores para consolas fijas vs las consolas portátiles y que en conjunto tienen mas ventas que para los videojuegos en PC.  

### Juegos por género

In [ ]:
sales_by_genre = df_games.groupby('genre')['total_sales'].sum().sort_values(ascending=False)


plt.figure(figsize=(12, 6))
sales_by_genre.plot(kind='bar', color='skyblue')
plt.title('Ventas de juegos por género')
plt.xlabel('Género')
plt.ylabel('Ventas (en millones)')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y')
plt.show()

Los géneros más rentables son: Action, sports y shooter con mas de mil millones en ventas, mientras que los géneros con menos ventas son; puzzle, adventure y strategy quienes apenas llegan a los 250 millones en ventas.

## Regiones

### Plataformas principales

In [ ]:
top_platforms_na = df_filtered.groupby('platform')['na_sales'].sum().nlargest(5)
top_platforms_eu = df_filtered.groupby('platform')['eu_sales'].sum().nlargest(5)
top_platforms_jp = df_filtered.groupby('platform')['jp_sales'].sum().nlargest(5)

print("Las cinco principales plataformas en Norteamérica:")
print(top_platforms_na)
print("\nLas cinco principales plataformas en Europa:")
print(top_platforms_eu)
print("\nLas cinco principales plataformas en Japón:")
print(top_platforms_jp)

1.- PS2: La plataforma líder en todas las regiones, pero su dominio es más pronunciado en Norteamérica, donde las ventas superan los 500 millones de dolares.
2.- GBA: Aunque es la segunda plataforma más vendida en Norteamérica, su cuota de mercado en Europa y Japón es mucho menor en comparación.
3.- DS: Indica una preferencia más fuerte por la DS en Europa y Japón es la segunda plataforma más vendida. Aunque ocupa el quinto lugar en Norteamérica.
4.- XB: Fue más popular en el mercado norteamericano en comparación con otras regiones, su presencia en Europa y Japón es mucho menos significativa.
5.- GC y PS: GameCube (GC) y PlayStation (PS) tienen una presencia notable en Japón por lo que se nota una preferencia por las plataformas de Nintendo y Sony en el mercado japonés ya que sus ventas en Norteamérica y Europa son relativamente bajas en comparación con otras plataformas.

### Géneros principales

In [ ]:
top_genres_na = df_filtered.groupby('genre')['na_sales'].sum().nlargest(5)
top_genres_eu = df_filtered.groupby('genre')['eu_sales'].sum().nlargest(5)
top_genres_jp = df_filtered.groupby('genre')['jp_sales'].sum().nlargest(5)

print("Los cinco géneros principales en Norteamérica:")
print(top_genres_na)
print("\nLos cinco géneros principales en Europa:")
print(top_genres_eu)
print("\nLos cinco géneros principales en Japón:")
print(top_genres_jp)

Para Norteamérica los juegos de deportes (Sports) y de acción (Action) lideran las ventas, seguidos de cerca por los juegos de carreras (Racing), plataformas (Platform) de manera similar que en Europa, mientras que en Japón los juegos de rol (Role-Playing) son los más vendidos. Estas diferencias reflejan las preferencias culturales y las tendencias de juego específicas de cada región.

### Clasificaciones de ESRB

Si las clasificaciones de ESRB afectan a las ventas en regiones individuales.

In [ ]:
region_sales_rating = df_filtered.groupby(['rating'])[['na_sales', 'eu_sales', 'jp_sales']].mean()

plt.figure(figsize=(10, 8))
region_sales_rating.plot(kind='bar', stacked=True, rot=45)
plt.title('Ventas promedio por clasificación de ESRB en cada región')
plt.xlabel('Clasificación de ESRB')
plt.ylabel('Ventas promedio (millones)')
plt.legend(['Norteamérica', 'Europa', 'Japón'])
plt.show()

Según la gráfica el mayor número de ventas lo encabeza la clasificación AO *(Adults Only/Adultos únicamente)* tanto para Norteamérica como para Europa, seguido aunque con algo de diferencia la clasificación M *(Mature/Maduro)*, lo que parece indicar que si afecta la clasificación de un juego en las ventas de estos dos países, sin embargo parece no ser el caso de Japón donde normalmente están mas restringidos este tipo de clasificaciones.

## Prueba de hipótesis

### Calificaciones para las plataformas Xbox One y PC

En esta sección nos interesa saber si las calificaciones promedio de los usuarios para las plataformas Xbox One y PC son las mismas por lo que partimos de las siguientes hipótesis:

**Hipótesis nula (H0)**: Las calificaciones promedio de los usuarios para las plataformas Xbox One y PC son iguales.

**Hipótesis alternativa (H1)**: Las calificaciones promedio de los usuarios para las plataformas Xbox One y PC no son iguales.

Con estas hipótesis en mente, podemos realizar la prueba estadística.

In [ ]:
ratings_xbox_one = df_games[df_games['platform'] == 'XOne']['user_score'].dropna()
ratings_pc = df_filtered[df_filtered['platform'] == 'PC']['user_score'].dropna()

alpha = 0.05 
results = st.ttest_ind(ratings_xbox_one, ratings_pc)

print('Valor p:', results.pvalue) 

if results.pvalue < alpha: 
    print("Rechazamos la hipótesis nula")
else:
    print("No podemos rechazar la hipótesis nula")

El valor p es 3.75e-05, lo que indica una diferencia muy significativa en las calificaciones promedio de los usuarios entre las plataformas Xbox One y PC.

Por lo tanto, podemos concluir que hay evidencia suficiente para sugerir que las calificaciones promedio de los usuarios para las plataformas Xbox One y PC no son iguales. Esto significa que hay diferencias significativas en cómo los usuarios califican los juegos en estas dos plataformas.

### Calificaciones para los géneros de Acción y Deportes 

En este caso, estamos interesados en determinar si hay diferencias significativas en las calificaciones promedio de los usuarios entre los géneros de Acción y Deportes, por lo que formulamos las siguientes hipótesis:

**Hipótesis nula (H0)**: Las calificaciones promedio de los usuarios para los géneros de Acción y Deportes son iguales.

**Hipótesis alternativa (H1)**: Las calificaciones promedio de los usuarios para los géneros de Acción y Deportes no son iguales.

In [ ]:
ratings_action = df_games[df_games['genre'] == 'Action']['user_score'].dropna()
ratings_sports = df_filtered[df_filtered['genre'] == 'Sports']['user_score'].dropna()

alpha = 0.05 
results = st.ttest_ind(ratings_action, ratings_sports)

print('Valor p:', results.pvalue) 

if results.pvalue < alpha: 
    print("Rechazamos la hipótesis nula")
else:
    print("No podemos rechazar la hipótesis nula")

Cuando el valor p es mayor que el nivel de significancia seleccionado (en este caso, alpha = 0.05), no tenemos suficiente evidencia para rechazar la hipótesis nula. Esto significa que no podemos concluir que hay diferencias significativas en las calificaciones promedio de los usuarios entre los géneros de Acción y Deportes.

## Conclusión general

Para este proyecto, comenzamos analizando y preparando los datos, luego realizamos análisis exploratorio y pruebas estadísticas para responder a varias preguntas relacionadas con las ventas de videojuegos.

1. Revisamos y limpiamos los datos, cambiando los nombres de las columnas a minúsculas, convirtiendo los tipos de datos, tratando los valores ausentes y creamos una columna adicional para las ventas totales de cada juego.

2. Investigamos la cantidad de juegos lanzados en diferentes años y examinamos cómo varían las ventas entre plataformas a lo largo del tiempo. Identificamos las plataformas líderes en ventas, observamos la evolución de las ventas de plataformas y analizamos las diferencias en las ventas entre géneros de juegos. También exploramos la relación entre las ventas y las reseñas de usuarios y profesionales.

3. Creamos perfiles para cada región, identificando las cinco principales plataformas y géneros. Observamos las variaciones en las cuotas de mercado entre ellas y las diferencias en los géneros principales. Además, visualizamos si las clasificaciones de ESRB afectan a las ventas en regiones individuales.

4. Formulamos y probamos dos hipótesis relacionadas con las calificaciones promedio de usuarios para diferentes plataformas y géneros de juegos. Utilizamos la prueba t de Student y establecimos un valor de alfa para determinar si había evidencia suficiente para rechazar la hipótesis nula en favor de la alternativa.

Basándonos en los resultados de nuestro análisis, podemos concluir que existen varias tendencias y patrones en la industria de los videojuegos que pueden ser útiles para la toma de decisiones en la tienda Ice. Por ejemplo, identificamos las plataformas y géneros más populares en diferentes regiones, así como la influencia de las reseñas en las ventas. Este proyecto nos proporcionó una visión general de la industria de los videojuegos y nos permitió identificar patrones importantes que pueden ayudar a la tienda Ice a planificar estrategias de marketing y selección de productos para el futuro.